In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import numpy.random as npr  
from numpy.random import default_rng, SeedSequence

### grille des sauts

In [6]:
def grille_sauts(T, kn,lamda):
    jumps_t = []
    grid = []
    x =  np.random.exponential(1/lamda)
    sum_x = x
    
    vec = []
    t = 0
    vec.append(t)
    vec.append(0)
    grid.append(vec)
    vec = []
    
    # construction of the jump times
    while sum_x < T:
        jumps_t.append(sum_x)
        x = np.random.exponential(lamda)
        sum_x += x
    
    i = 0
    # construction of the grid on the interval [0,last jump]
    jump_size = len(jumps_t)
    epsilon = 0
    while t < T and i < jump_size:
        epsilon = 0
        t = min(t + 1.0/kn, jumps_t[i])
        if t == jumps_t[i]:
            i += 1
            epsilon = 1 # it is a jump time
        
        vec.append(t)
        vec.append(epsilon)
        grid.append(vec)
        
        vec = []
    
    # to construct the grid on the inverval ]last jump, T]
    while t < T:
        t += 1.0/kn
        
        vec.append(t)
        vec.append(0) # epsilon = 0
        grid.append(vec)
        
        vec = []
    
    vec = []
    vec.append(T)
    vec.append(0)
    
    if grid[-1][0] < T:
        grid.append(vec)
    
    if grid[-1][0] > T:
        grid.pop()
        grid.append(vec)
    
    return grid


# mesure infinie

In [7]:
b = 0.01
c = 0.04
un = 10 ** 4
bn = b
alpha_1 = 1.3
alpha_2 = 2.0 / 3.0
theta_n_1 = (2.0 * un ** alpha_1) / alpha_1
wn_1 = (2.0 - alpha_1) / 2.0 * un ** ((2.0 - alpha_1) / 2.0)
theta_n_2 = (2.0 * un ** alpha_2 - 1.0) / alpha_2
wn_2 = (2.0 * un ** (alpha_2 - 2.0) / (2.0 - alpha_2)) ** (-1.0 / 2.0)


### call

In [23]:
import math
import random

def k_n(theta_n, bn, c, w_n):
    if min(theta_n / (bn ** 2), math.sqrt(theta_n) / c) >= w_n:
        return 1
    else:
        return max(bn ** 2 * w_n, (c * w_n) ** 2)

def inverted_h_premiere_mesure_infinie(u, alpha, theta_n, un):
    if u < 1 / 2:
        return -((2 * u) ** (-1 / alpha)) / un
    else:
        return ((2 - alpha * theta_n * u / un ** alpha) ** (-1 / alpha)) / un

def inverted_h_seconde_mesure_infinie(u, alpha, theta_n, un):
    if u < (1 / (alpha * theta_n)) * (un ** alpha - 1):
        return -((alpha * theta_n * u + 1) ** (-1 / alpha))
    else:
        return ((-alpha * theta_n * u + 2 * un ** alpha - 1) ** (-1 / alpha))
    
def mc_pricer_euler_scheme_call_1(M, strike, x0, T, kn, un, c, bn, alpha, theta_n):
    result = []
    grid = []
    interm=[]
    x = x0
    
    for j in range(1, M):
        h = 0
        saut = 0
        x = x0
        grid = grille_sauts(T, kn, theta_n)        
        for i in range(1, len(grid)):
            z=np.random.normal(0, 1)
            u=np.random.uniform(0,1)
            h = grid[i][0] - grid[i-1][0]
            var_norm = z * math.sqrt(c * h)
            x = x * (1 + bn * h + var_norm)
            if grid[i][1] != 0:
                saut = inverted_h_premiere_mesure_infinie(random.uniform(0, 1), alpha_1, theta_N_1, uN)
                x = x * (1 + saut)
                
        interm.append(max(0, x - strike))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

def mc_pricer_euler_scheme_call_2(M, strike, x0, T, kn, un, c, bn, alpha, theta_n):
    result = []
    grid = []
    interm=[]
    x = x0
    
    for j in range(1, M):
        h = 0
        saut = 0
        x = x0
        grid = grille_sauts(T, kn, theta_n)        
        for i in range(1, len(grid)):
            z=np.random.normal(0, 1)
            u=np.random.uniform(0,1)
            h = grid[i][0] - grid[i-1][0]
            var_norm = z * math.sqrt(c * h)
            x = x * (1 + bn * h + var_norm)
            if grid[i][1] != 0:
                saut = inverted_h_seconde_mesure_infinie(random.uniform(0, 1), alpha_1, theta_N_1, uN)
                x = x * (1 + saut)
                
        interm.append(max(0, x - strike))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

In [38]:
M = 1000  # Number of paths
x0 = 100  # Spot
T = 1
uN = pow(10, 4)
alpha_1 = 1.5
alpha_2 = 1.8
bn = 1
c = 1
theta_N_1 = (2.0 * pow(uN, alpha_1)) / alpha_1
wN_1 = np.sqrt((2.0 - alpha_1) / 2.0) * pow(uN, (2.0 - alpha_1) / 2.0)
theta_N_2 = (2.0 * pow(uN, alpha_2) - 1.0) / alpha_2
wN_2 = pow(2.0 * pow(uN, alpha_2 - 2.0) / (2.0 - alpha_2), -1.0 / 2.0)
kN_1 = k_n(theta_N_1, bn, c, wN_1)
kN_2 = k_n(theta_N_2, bn, c, wN_2)

strike=80
kn = 1000


invertedh=inverted_h_premiere_mesure_infinie(uN, alpha_1, theta_N_1, uN)

print(mc_pricer_euler_scheme_call_1(M, strike, x0, T, kN_1, uN, c, bn, alpha_1, theta_N_1))

{'mean': 126.99371310480663, 'std': 91.1946300674981, 'lower': 121.34141137356715, 'upper': 132.6460148360461}


In [39]:
print(mc_pricer_euler_scheme_call_2(M, strike, x0, T, kN_1, uN, c, bn, alpha_1, theta_N_1))

{'mean': 127.40661854065135, 'std': 87.7981639071966, 'lower': 121.96483196303177, 'upper': 132.84840511827093}


### log-contrat

In [16]:
def mc_pricer_euler_scheme_logcontrat(M, strike, x0, T, kn, un, c, bn, alpha, theta_n, invertedh):
    result = []
    x = x0
    saut = 0
    interm=[]
    grid_taille = 0
    
    for j in range(1, M):
        h = 0
        saut = 0
        x = x0
        grid = grille_sauts(T, kn, theta_n)
        grid_taille = len(grid)
        
        for i in range(1, grid_taille):
            z=np.random.normal(0, 1)
            u=np.random.uniform(0,1)
            h = grid[i][0] - grid[i-1][0]
            var_norm = z * math.sqrt(c * h)
            x = x * (1 + bn * h + var_norm)
            if grid[i][1] != 0:
                saut = inverted_h_seconde_mesure_infinie(random.uniform(0, 1), alpha_1, theta_N_1, uN)
                x = x * (1 + saut)
                
        interm.append(-2 * np.log(x / x0))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

In [17]:
M = 1000  # Number of paths
x0 = 100  # Spot
T = 1
uN = pow(10, 4)
alpha_1 = 1.5
alpha_2 = 1.8
bn = 1
c = 1
theta_N_1 = (2.0 * pow(uN, alpha_1)) / alpha_1
wN_1 = np.sqrt((2.0 - alpha_1) / 2.0) * pow(uN, (2.0 - alpha_1) / 2.0)
theta_N_2 = (2.0 * pow(uN, alpha_2) - 1.0) / alpha_2
wN_2 = pow(2.0 * pow(uN, alpha_2 - 2.0) / (2.0 - alpha_2), -1.0 / 2.0)
kN_1 = k_n(theta_N_1, bn, c, wN_1)
kN_2 = k_n(theta_N_2, bn, c, wN_2)

strike = 120
kn = 1000


invertedh=inverted_h_premiere_mesure_infinie(uN, alpha_1, theta_N_1, uN)

mc_pricer_euler_scheme_logcontrat(M, strike, x0, T, kN_1, uN, c, bn, alpha_1, theta_N_1, invertedh)

C:\Users\MSI\AppData\Local\Temp/ipykernel_1472/2868153095.py:25: RuntimeWarning: invalid value encountered in log
  interm.append(-2 * np.log(x / x0))


{'mean': nan, 'std': nan, 'lower': nan, 'upper': nan}